<a href="https://colab.research.google.com/github/NayeemIMI/Data-Encryption-with-RSA-Algorithm/blob/main/RSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from random import randrange, getrandbits
from itertools import repeat
from functools import reduce
 
def getPrime(n):
    "Get a n-bit pseudo-random prime"
    def isProbablePrime(n, t = 7):
        def isComposite(a):
            if pow(a, d, n) == 1:
                return False
            for i in range(s):
                if pow(a, 2 ** i * d, n) == n - 1:
                    return False
            return True
     
        assert n > 0
        if n < 3:
            return [False, False, True][n]
        elif not n & 1:
            return False
        else:
            s, d = 0, n - 1
            while not d & 1:
                s += 1
                d >>= 1
        for _ in repeat(None, t):
            if isComposite(randrange(2, n)):
                return False
        return True   
     
    p = getrandbits(n)
    while not isProbablePrime(p):
        p = getrandbits(n)
    return p
 
def inv(p, q):
    """Multiplicative inverse"""
    def xgcd(x, y):
        """Extended Euclidean Algorithm"""
        s1, s0 = 0, 1
        t1, t0 = 1, 0
        while y:
            q = x // y
            x, y = y, x % y
            s1, s0 = s0 - q * s1, s1
            t1, t0 = t0 - q * t1, t1
        return x, s0, t0      
 
    s, t = xgcd(p, q)[0:2]
    assert s == 1
    if t < 0:
        t += q
    return t
 
def genRSA(p, q):
    """Generate public and private keys"""
    phi, mod = (p - 1) * (q - 1), p * q
    if mod < 65537:
        return (3, inv(3, phi), mod)
    else:
        return (65537, inv(65537, phi), mod)    
 
def text2Int(text):
    """Convert a text string into an integer"""
    return reduce(lambda x, y : (x << 8) + y, map(ord, text))
 
def int2Text(number, size):
    """Convert an integer into a text string"""
    text = "".join([chr((number >> j) & 0xff)
                    for j in reversed(range(0, size << 3, 8))])
    return text.lstrip("\x00")
 
def int2List(number, size):
    """Convert an integer into a list of small integers"""
    return [(number >> j) & 0xff
            for j in reversed(range(0, size << 3, 8))]
 
def list2Int(listInt):
    """Convert a list of small integers into an integer"""
    return reduce(lambda x, y : (x << 8) + y, listInt)
 
def modSize(mod):
    """Return length (in bytes) of modulus"""
    modSize = len("{:02x}".format(mod)) // 2
    return modSize
 
def encrypt(ptext, pk, mod):
    """Encrypt message with public key"""
    size = modSize(mod)
    output = []
    while ptext:
        nbytes = min(len(ptext), size - 1)
        aux1 = text2Int(ptext[:nbytes])
        assert aux1 < mod
        aux2 = pow(aux1, pk, mod)
        output += int2List(aux2, size + 2)
        ptext = ptext[size:]
    return output
 
def decrypt(ctext, sk, p, q):
    """Decrypt message with private key
    using the Chinese Remainder Theorem"""
    mod = p * q
    size = modSize(mod)
    output = ""
    while ctext:
        aux3 = list2Int(ctext[:size + 2])
        assert aux3 < mod
        m1 = pow(aux3, sk % (p - 1), p)
        m2 = pow(aux3, sk % (q - 1), q)
        h = (inv(q, p) * (m1 - m2)) % p
        aux4 = m2 + h * q
        output += int2Text(aux4, size)
        ctext = ctext[size + 2:]
    return output
 
if __name__ == "__main__":
 
    from math import log10
    from time import time
 
    def printHexList(intList):
        """Print ciphertext in hex"""
        for index, elem in enumerate(intList):
            if index % 32 == 0:
                print()            
            print("{:02x}".format(elem), end = "")
        print("\n")
 
    def printLargeInteger(number):
        """Print long primes in a formatted way"""
        string = "{:02x}".format(number)
        for j in range(len(string)):
            if j % 64 == 0:
                print()
            print(string[j], end = "")
        print("\n")
 
    def testCase(p, q, msg):
        """Execute test case: generate keys, encrypt message and
           decrypt resulting ciphertext"""
        #print("Key size: {:0d} bits".format(round(log10(p * q) / log10(2))))
        print("\nPrime #1:", end = "")
        printLargeInteger(p)
        print("Prime #2:", end = "")
        printLargeInteger(q)
        print("Plaintext:", msg,"\n")
        pk, sk, mod = genRSA(p, q)
        print("Public key: ",pk,"\n")
        print("Private key: ",end = "")
        printLargeInteger(sk)
        ctext = encrypt(msg, pk, mod)
        print("Ciphertext:", end = "")
        printHexList(ctext)
        ptext = decrypt(ctext, sk, p, q)
        print("Recovered plaintext:", ptext, "\n")
    keysize=int(input("Enter the key size in Bits:"))
    p1 = getPrime(keysize)
    p2 = getPrime(keysize)
    msg=input("Enter text: ")
    testCase(p1, p2, msg)

Enter the key size in Bits:1024
Enter text: Nayeem

Prime #1:
8054b88ee660474c6b34a2b50c7806f373f60fef91c5174fef6dc826a08c91ea
86e016578f23125d4fe5ffced43fd82a767308c0952a3b38247e63b98d6dd33a
8838d1d74e788c8511b6cfa178fe9fbf6a990262e9622ae1b57df4aa211a75a2
0a3e4a401869285562af4e5042a6136ff49a4cc3b726645a835240d78634ad25

Prime #2:
31b750edaaa64e318fbe35ae3865af6c2f4111a59209d2a082a55a495f55867b
1ee730a3a6969be86bda73e19dce238e7f2ba46f0494dd22f5dcb1c1099d169a
c9d0e27f3eb2cdfabc4811210e4a864e02de0773f64eef1c40cd1da560d7c46b
2819e5c49620ac1a80544f34d6ff8b9f715f8f6b6807400334d9bbc75ff1be53

Plaintext: Nayeem 

Public key:  65537 

Private key: 
5fdc4fd8943285ad38e8e719ce692d5957f0361d86d939f55dbb9a04b4217354
b5cb3f170f278412a9f068be411e3e3d075bfa0aaf0acec984be9d5491934b3c
a4ccf15928a4f2827fb112202c74b9661dffc4978a6da24150a4a07c3a910aba
d56832a1292a4318c888656763622f5061e646a9c4cd621a3b02de5a3a2a8647
b848d574413fcc8013b66ef6a85c15881730d8277f6ea07a53d9ae7266617ced
5222d4b27ad7bd6c4f67e95fe4